# Note
This notebook can be run on google colab for improved performance. The code changes necessary for running on this system are commented over the code.

## Data preprocessing

In [ ]:
! pip install \
  scprep\
  spacy==2.3.2 \
  sentence_transformers==0.4.0 \
  phate==1.0.4 && \
  python -m spacy download es_core_news_lg

In [ ]:
# Define branch to clone
! branch_name='#50_dfq_sbert_fine_tuning' && \
  git clone --branch $branch_name https://github.com/wri-dssg/policy-data-analyzer.git

In [ ]:
import pandas as pd
import sys
import os
from sklearn.model_selection import train_test_split
from sentence_transformers import SentencesDataset, SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from torch import nn, Tensor
from typing import Iterable, Dict
from torch.utils.data import DataLoader
import math

os.chdir("policy-data-analyzer")
from tasks.data_loader.src.utils import *
from tasks.data_augmentation.src.zero_shot_classification.latent_embeddings_classifier import *
from tasks.evaluate_model.src.model_evaluator import *
from tasks.data_visualization.src.plotting import *

from google.colab import drive
drive.mount('/content/drive')

## Fine-tuning the embedding model on the labeled data

### Something we can try out:
https://www.sbert.net/examples/training/data_augmentation/README.html#extend-to-your-own-datasets

### Links:
https://github.com/UKPLab/sentence-transformers/issues/350

https://omoindrot.github.io/triplet-loss

### Possible tasks for fine-tuning:
1) Given a pair of sentence embeddings, do they belong to the same category (binary)?

2) Given a sentence and a category embedding, does the sentence belong to the category (binary)?

3) Given a sentence embedding, use a classifier to predict its category (multiclass) [https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/nli/training_nli.py](https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/nli/training_nli.py)

4) Use a triplet loss approach such that sentences (texts) that have the same labels will become close in vector space, while sentences with a different label will be further away [https://github.com/UKPLab/sentencetransformers/blob/master/examples/training/other/training_batch_hard_trec_continue_training.py](https://github.com/UKPLab/sentencetransformers/blob/master/examples/training/other/training_batch_hard_trec_continue_training.py)
   
#### In this notebook **task number 3** is used to fine-tune the model.

In [ ]:
# Train test split stratified
X_train, X_test, y_train, y_test = train_test_split(all_sents, all_labels, test_size=0.15, stratify=all_labels, random_state=42)

In [ ]:
# Define model to fine-tune
model = SentenceTransformer('stsb-xlm-r-multilingual')
# model = SentenceTransformer('xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
class SoftmaxClassifier(nn.Module):
    """
    This loss adds a softmax classifier on top of the output of the transformer network. 
    It takes a sentence embedding and learns a mapping between it and the corresponding category.
    :param model: SentenceTransformer model
    :param sentence_embedding_dimension: Dimension of your sentence embeddings
    :param num_labels: Number of different labels
    """
    def __init__(self,
                 model: SentenceTransformer,
                 sentence_embedding_dimension: int,
                 num_labels: int):
        super(SoftmaxClassifier, self).__init__()
        self.model = model
        self.num_labels = num_labels
        self.classifier = nn.Linear(sentence_embedding_dimension, num_labels)

    def forward(self, sentence_features: Iterable[Dict[str, Tensor]], labels: Tensor):
        # Get batch sentence embeddings
        features = self.model(sentence_features[0])['sentence_embedding']
        
        # Get batch loss
        output = self.classifier(features)
        loss_fct = nn.CrossEntropyLoss()

        if labels is not None:
            loss = loss_fct(output, labels.view(-1))
            return loss
        else:
            return features, output

In [ ]:
# Load data samples into batches
train_batch_size = 16
label2int = dict(zip(label_names, range(len(label_names))))
train_samples = []
for sent, label in zip(X_train, y_train):
    label_id = label2int[label]
    train_samples.append(InputExample(texts=[sent], label=label_id))
train_dataset = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

# Define the way the loss is computed
classifier = SoftmaxClassifier(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=len(label2int))

# Configure the dev set evaluator - still need to test whether this works
dev_samples = []
for sent, label in zip(X_test, y_test):
    label_id = label2int[label]
    dev_samples.append(InputExample(texts=[sent], label=label_id))
dev_dataset = SentencesDataset(dev_samples, model=model)
dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=train_batch_size)
dev_evaluator = LabelAccuracyEvaluator(dataloader=dev_dataloader, softmax_model=classifier, name='lae-dev')

In [ ]:
# Configure the training
num_epochs = 1
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1)  # 10% of train data for warm-up
model_save_path = "../../output/FineTuning"
# model_save_path = "/content/drive/MyDrive/WRI-LatinAmerica-Talent/Modeling/FineTuning"

In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, classifier)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

In [ ]:
# Load the saved model and obtain random sentence embedding
load_model = SentenceTransformer(model_save_path)
load_model.encode(all_sents[0])

## Run fine tuning experiments

In [ ]:
class SoftmaxClassifier(nn.Module):
    """
    This loss adds a softmax classifier on top of the output of the transformer network. 
    It takes a sentence embedding and learns a mapping between it and the corresponding category.
    :param model: SentenceTransformer model
    :param sentence_embedding_dimension: Dimension of your sentence embeddings
    :param num_labels: Number of different labels
    """
    def __init__(self,
                 model: SentenceTransformer,
                 sentence_embedding_dimension: int,
                 num_labels: int):
        super(SoftmaxClassifier, self).__init__()
        self.model = model
        self.num_labels = num_labels
        self.classifier = nn.Linear(sentence_embedding_dimension, num_labels)

    def forward(self, sentence_features: Iterable[Dict[str, Tensor]], labels: Tensor):
        # Get batch sentence embeddings
        features = self.model(sentence_features[0])['sentence_embedding']
        
        # Get batch loss
        output = self.classifier(features)
        loss_fct = nn.CrossEntropyLoss()

        if labels is not None:
            loss = loss_fct(output, labels.view(-1))
            return loss
        else:
            return features, output

Reading from Excel. <span style="color:red"><strong>WARNING!</strong></span> This path is deprecated. The two cells below should not be run from this version on

In [ ]:
# Reading data from excel
data_excel = pd.read_excel("/content/drive/MyDrive/WRI-LatinAmerica-Talent/Cristina_Policy_Files/WRI_Policy_Tags.xlsx", engine="openpyxl", sheet_name=None)

In [ ]:
# Formatting the data
all_labeled_sentences = country_labeled_sentences(data_excel)
labeled_sents = dict()
for sents in all_labeled_sentences.values():
    labeled_sents.update(sents)

# Fitlering out General Incentive and Unknown sentences
filtered_sents_maps = [sent for sent in labeled_sents.values() if sent['labels'][0] not in ["General incentive", "Unknown", "Other"]]
all_sents = [sent['text'] for sent in filtered_sents_maps]
all_labels = [sent['labels'][0] for sent in filtered_sents_maps]
all_labels = merge_labels(all_labels, ["Credit", "Guarantee"])
label_names = list(set(all_labels))
numeric_labels = labels2numeric(all_labels, label_names)
label_names

Reading files from JSON

In [9]:
# This is the dictionary used to transform policy instrument labels into numeric codes. BEWARE that we have a new category which is 0. This new category might represent unknown incentive or no incentive!!!
policy_dict = {
'Technical assistance' : 6, 
'Supplies' : 4, 
'Credit' : 1, 
'Fine' : 3, 
'Guarantee' : 1, 
'Tax deduction' : 5,
'Direct payment' : 2, 
'General' : 0, 
'Unknown' : 0, 
'Other' : 0, 
'Nan' : 0 }

# This is the dictionary used to transform is_incentive labels into numeric codes.
incentive_dict = {
'Incentive' : 1, 
'Disincentive' : 1, 
'Unknown' : 0, 
'Nan' : 0}

In [6]:
# This function has been moved to utils.py and can be removed
def select_labels(dataset, labels_to_be_retrieved):
  new_dict = {}
  for key, value in dataset.items():
    if value['labels'] in labels_to_be_retrieved:
      new_dict[key] = value
  return new_dict

In [9]:
#THIS IS THE PREFERRED PATH
# If the json is in the format that includes headers and other titles:
#TODO: Adapt/choose the path according to your system configuration

# dataset_fname = "C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data"
# dataset_fname = "/content/drive/MyDrive/WRI-LatinAmerica-Talent/Cristina_Policy_Files/Tagged_sentence_lists/Rater_3_labeled.json"
dataset_fname = "/content/drive/MyDrive/Official Folder of WRI Latin America Project/WRI-LatinAmerica-Talent/Cristina_Policy_Files/Tagged_sentence_lists/Rater_3_labeled.json"

dataset = load_file(dataset_fname)
dataset_map = labeled_sentences_from_dataset(dataset) # Labels AND sentences
dataset_map_target_labels =  select_labels(dataset_map, [1,2,3,4,5,6]) # Adjust to the labels you want to use for your analysis. Remember there are 7 labels from 0 to 6.
all_sents = sentences_from_dataset(dataset) # Just sentences
all_labels = labels_from_dataset(dataset, "labels") # Just labels. Use "labels" if you want to retrieve policy instrument tags
                                                    # Use "incentive" if you want to retrieve is_incentive tags.

In [ ]:
# If the json is in the format where it only contains sentences and labels
#TODO: Adapt/choose the path according to your system configuration

dataset_fname = "/content/drive/MyDrive/Official Folder of WRI Latin America Project/WRI-LatinAmerica-Talent/Cristina_Policy_Files/Tagged_sentence_lists/Rater_3_labeled_sentences.json"
dataset_fname = "/content/drive/MyDrive/WRI-LatinAmerica-Talent/Cristina_Policy_Files/Tagged_sentence_lists/Rater_3_labeled_sentences.json"


dataset = load_file(dataset_fname) 
dataset_map = labels_from_model_output(dataset) # Labels AND sentences
all_sents = sentences_from_model_output(dataset) # Just sentences
all_labels = labels_from_model_output(dataset, "labels") # Just labels. Use "labels" if you want to retrieve policy instrument tags
                                                    # Use "incentive" if you want to retrieve is_incentive tags.

In [ ]:
# WARNING! Deprecated using the current labeled data already in numeric formate!
# The rest:
label_names = unique_labels(all_labels)
numeric_labels = labels2numeric(all_labels, label_names)
label_names

In [ ]:
import time
import cupy

In [ ]:
import spacy
spacy.prefer_gpu()
es_nlp = spacy.load('es_core_news_lg')

In [ ]:
model_names = ['distiluse-base-multilingual-cased-v2', 'stsb-xlm-r-multilingual', 'paraphrase-xlm-r-multilingual-v1', 'quora-distilbert-multilingual']

# Train test split stratified
all_test_perc = [0.15, 0.2, 0.25, 0.3]

# Output setup
output = {}

for test_perc in all_test_perc:
  output[f"test_perc={test_perc}"] = {}
  X_train, X_test, y_train, y_test = train_test_split(all_sents, all_labels, test_size=test_perc, stratify=all_labels, random_state=69420)

  # Load data samples into batches
  train_batch_size = 16
  label2int = dict(zip(label_names, range(len(label_names))))
  train_samples = []
  for sent, label in zip(X_train, y_train):
      label_id = label2int[label]
      train_samples.append(InputExample(texts=[sent], label=label_id))

  # Configure the dev set evaluator - still need to test whether this works
  dev_samples = []
  for sent, label in zip(X_test, y_test):
      label_id = label2int[label]
      dev_samples.append(InputExample(texts=[sent], label=label_id))
  
  for model_name in model_names:
    # Setup
    model_preds = []
    model_scores = []
    output[f"test_perc={test_perc}"][model_name] = []
    
    # Train set config
    model = SentenceTransformer(model_name)
    train_dataset = SentencesDataset(train_samples, model=model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    
    # Define the way the loss is computed
    classifier = SoftmaxClassifier(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=len(label2int))
    
    # Dev set config
    dev_dataset = SentencesDataset(dev_samples, model=model)
    dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=train_batch_size)
    dev_evaluator = LabelAccuracyEvaluator(dataloader=dev_dataloader, softmax_model=classifier, name='lae-dev')

    # Configure the training
    max_num_epochs = 10
        
    for num_epochs in range(2, max_num_epochs + 2, 2):
        print("Num epochs:", num_epochs)
        
        warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1)  # 10% of train data for warm-up
        model_deets = f"model={model_name}_test-perc={test_perc}_n-epoch={num_epochs}"
        model_save_path = f"/content/drive/MyDrive/WRI-LatinAmerica-Talent/Modeling/FineTuning_{model_deets}"
        

        # Train the model
        start = time.time()
        model.fit(train_objectives=[(train_dataloader, classifier)],
                  evaluator=dev_evaluator,
                  epochs=2, # We always tune on an extra epoch to see the performance gain
                  evaluation_steps=1000,
                  warmup_steps=warmup_steps,
                  output_path=model_save_path
                  )
        
        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Time taken for fine-tuning:", "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        
        ### Classify sentences
        # Projection matrix Z low-dim projection
        print("Classifying sentences...")
        proj_matrix = cupy.asnumpy(calc_proj_matrix(all_sents, 50, es_nlp, model, 0.01))
        all_sent_embs = encode_all_sents(all_sents, model, proj_matrix)
        all_label_embs = encode_labels(label_names, model, proj_matrix)
        visualize_embeddings_2D(np.vstack(all_sent_embs), all_labels, tsne_perplexity=50, store_name=f"{model_save_path}/{model_deets}")
        model_preds, model_scores = calc_all_cos_similarity(all_sent_embs, all_label_embs, label_names)
        
        ### Evaluate the model
        numeric_preds = labels2numeric(model_preds, label_names)
        evaluator = ModelEvaluator(label_names, y_true=numeric_labels, y_pred=numeric_preds)
        
        output[f"test_perc={test_perc}"][model_name].append({"num_epochs": num_epochs, "avg_f1": evaluator.avg_f1.tolist()})
        
        evaluator.plot_confusion_matrix(color_map='Blues', exp_name=f"{model_save_path}/{model_deets}")

In [ ]:
output.keys()

dict_keys(['distiluse-base-multilingual-cased-v2', 'stsb-xlm-r-multilingual', 'paraphrase-xlm-r-multilingual-v1', 'quora-distilbert-multilingual'])

In [ ]:
new_json = {}

for key in output.keys():
  new_json[key] = {}
  for subkey in output[key].keys():
    new_json[key][subkey] = []
    for element in output[key][subkey]:
      el_copy = {"avg_f1": element["avg_f1"].tolist(), "num_epochs": element["num_epochs"]}
      new_json[key][subkey].append(el_copy)

In [ ]:
import json
with open("/content/drive/MyDrive/WRI-LatinAmerica-Talent/Modeling/FineTuningResults.json", "w") as f:
  json.dump(new_json, f)